<a href="https://colab.research.google.com/github/davidlealo/sic_ai_2025_sept/blob/main/6_proyectos/clase_40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pretty_midi music21 torch numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 31.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.6 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.11-py3-none-any.whl size=5595886 sha256=cf02b495786575de392d8d8ab638af8827675630f6442e6810536128cd8b653b
  Stored in directory: /root/.cache/pip/wheels/f4/ad/93/a7042fe12668827574927ade9deec7f29aad2a1001b1501882
Successfully built pretty_midi


In [3]:
import pretty_midi
import numpy as np
import torch
import torch.nn as nn
from music21 import stream, note, meter, key
import os

# 1. Definir una red neuronal simple para procesar notas MIDI (clasificación básica)
class MusicNet(nn.Module):
    def __init__(self, input_size=4, hidden_size=128, output_size=88):  # 88 teclas de piano
        super(MusicNet, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Último paso de la secuencia
        return torch.sigmoid(out)

# 2. Función para cargar y procesar el archivo MIDI
def load_midi(file_path):
    try:
        midi_data = pretty_midi.PrettyMIDI(file_path)
        notes = []
        for instrument in midi_data.instruments:
            if not instrument.is_drum:  # Ignorar pistas de batería
                for n in instrument.notes:
                    notes.append({
                        'pitch': n.pitch,
                        'start': n.start,
                        'duration': n.end - n.start,
                        'velocity': n.velocity
                    })
        return notes, midi_data.get_tempo_changes()[1][0]  # Notas y tempo inicial
    except Exception as e:
        print(f"Error al cargar MIDI: {e}")
        return [], 120  # Tempo por defecto si falla

# 3. Preprocesar notas para el modelo de deep learning
def preprocess_notes(notes):
    # Convertir notas a un tensor [pitch, start, duration, velocity]
    data = np.array([[n['pitch'], n['start'], n['duration'], n['velocity']] for n in notes])
    # Normalizar datos (escala entre 0 y 1 para pitch y velocity, tiempos relativos)
    data[:, 0] = (data[:, 0] - 21) / (108 - 21)  # Normalizar pitch (rango de piano MIDI: 21-108)
    data[:, 3] = data[:, 3] / 127  # Normalizar velocity (0-127)
    data[:, 1] = data[:, 1] / np.max(data[:, 1]) if np.max(data[:, 1]) > 0 else 0  # Normalizar tiempos
    data[:, 2] = data[:, 2] / np.max(data[:, 2]) if np.max(data[:, 2]) > 0 else 0
    return torch.tensor(data, dtype=torch.float32).unsqueeze(0)  # Añadir dimensión de batch

# 4. Generar partitura con music21
def create_score(notes, tempo, output_pdf="output.pdf", output_xml="output.xml"):
    score = stream.Score()
    part = stream.Part()
    part.append(key.Key('C'))  # Tonalidad por defecto (ajusta según necesidad)
    part.append(meter.TimeSignature('4/4'))  # Compás por defecto
    part.append(tempo.MetronomeMark(number=tempo))  # Añadir tempo

    for n in notes:
        # Convertir duración a quarterLength (aproximada, basada en tempo)
        quarter_length = n['duration'] * (tempo / 60)  # Convertir segundos a negras
        if quarter_length < 0.1:  # Evitar duraciones demasiado cortas
            quarter_length = 0.25
        midi_note = note.Note(pitch=n['pitch'], quarterLength=quarter_length)
        midi_note.volume.velocity = n['velocity']
        part.append(midi_note)

    score.append(part)

    # Guardar como MusicXML y PDF
    score.write('musicxml', output_xml)
    score.write('musicxml.pdf', output_pdf)  # Requiere MuseScore instalado
    print(f"Partitura guardada como {output_pdf} y {output_xml}")

# 5. Función principal
def transcribe_midi_to_score(midi_file_path, output_pdf="output.pdf", output_xml="output.xml"):
    # Cargar MIDI
    notes, tempo = load_midi(midi_file_path)
    if not notes:
        print("No se encontraron notas en el archivo MIDI.")
        return

    # Preprocesar para el modelo
    input_data = preprocess_notes(notes)

    # Inicializar modelo (esto es un ejemplo; en producción, carga un modelo entrenado)
    model = MusicNet()
    model.eval()
    with torch.no_grad():
        output = model(input_data)
        # Aquí el modelo podría clasificar notas o corregirlas; por ahora, usamos las notas originales
        # En un caso real, usarías las predicciones para refinar notas, detectar acordes, etc.

    # Generar partitura con las notas originales (o procesadas por el modelo)
    create_score(notes, tempo, output_pdf, output_xml)

# 6. Ejecutar el programa
if __name__ == "__main__":
    # Reemplaza 'input.mid' con la ruta a tu archivo MIDI
    midi_file = "input.mid"
    if os.path.exists(midi_file):
        transcribe_midi_to_score(midi_file, output_pdf="partitura.pdf", output_xml="partitura.xml")
    else:
        print("Por favor, proporciona un archivo MIDI válido (input.mid).")

AttributeError: 'numpy.float64' object has no attribute 'MetronomeMark'

In [5]:
import pretty_midi
import numpy as np
import torch
import torch.nn as nn
from music21 import stream, note, meter, key, tempo  # Importa el módulo tempo
import os

# 1. Definir una red neuronal simple para procesar notas MIDI (clasificación básica)
class MusicNet(nn.Module):
    def __init__(self, input_size=4, hidden_size=128, output_size=88):  # 88 teclas de piano
        super(MusicNet, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Último paso de la secuencia
        return torch.sigmoid(out)

# 2. Función para cargar y procesar el archivo MIDI
def load_midi(file_path):
    try:
        midi_data = pretty_midi.PrettyMIDI(file_path)
        notes = []
        for instrument in midi_data.instruments:
            if not instrument.is_drum:  # Ignorar pistas de batería
                for n in instrument.notes:
                    notes.append({
                        'pitch': n.pitch,
                        'start': n.start,
                        'duration': n.end - n.start,
                        'velocity': n.velocity
                    })
        # Obtener tempo inicial de forma segura
        tempos = midi_data.get_tempo_changes()[1]
        bpm = float(tempos[0]) if len(tempos) > 0 else 120.0
        return notes, bpm
    except Exception as e:
        print(f"Error al cargar MIDI: {e}")
        return [], 120.0  # Tempo por defecto si falla

# 3. Preprocesar notas para el modelo de deep learning
def preprocess_notes(notes):
    # Convertir notas a un tensor [pitch, start, duration, velocity]
    data = np.array([[n['pitch'], n['start'], n['duration'], n['velocity']] for n in notes])
    # Normalizar datos (escala entre 0 y 1 para pitch y velocity, tiempos relativos)
    data[:, 0] = (data[:, 0] - 21) / (108 - 21)  # Normalizar pitch (rango de piano MIDI: 21-108)
    data[:, 3] = data[:, 3] / 127  # Normalizar velocity (0-127)
    data[:, 1] = data[:, 1] / np.max(data[:, 1]) if np.max(data[:, 1]) > 0 else 0  # Normalizar tiempos
    data[:, 2] = data[:, 2] / np.max(data[:, 2]) if np.max(data[:, 2]) > 0 else 0
    return torch.tensor(data, dtype=torch.float32).unsqueeze(0)  # Añadir dimensión de batch

# 4. Generar partitura con music21
def create_score(notes, bpm, output_pdf="output.pdf", output_xml="output.xml"):
    score = stream.Score()
    part = stream.Part()
    part.append(key.Key('C'))  # Tonalidad por defecto (ajusta según necesidad)
    part.append(meter.TimeSignature('4/4'))  # Compás por defecto
    part.append(tempo.MetronomeMark(number=bpm))  # Usa el módulo tempo (no la variable)

    for n in notes:
        # Convertir duración a quarterLength (aproximada, basada en bpm)
        quarter_length = n['duration'] * (bpm / 60)  # Convertir segundos a negras
        if quarter_length < 0.1:  # Evitar duraciones demasiado cortas
            quarter_length = 0.25
        midi_note = note.Note(pitch=n['pitch'], quarterLength=quarter_length)
        midi_note.volume.velocity = n['velocity']
        part.append(midi_note)

    score.append(part)

    # Guardar como MusicXML y PDF
    score.write('musicxml', output_xml)
    try:
        score.write('musicxml.pdf', output_pdf)  # Requiere MuseScore instalado
        print(f"Partitura guardada como {output_pdf} y {output_xml}")
    except Exception as e:
        print(f"Error al generar PDF: {e}. Asegúrate de que MuseScore esté instalado y configurado.")
        print(f"MusicXML guardado como {output_xml}")

# 5. Función principal
def transcribe_midi_to_score(midi_file_path, output_pdf="output.pdf", output_xml="output.xml"):
    # Cargar MIDI
    notes, bpm = load_midi(midi_file_path)
    if not notes:
        print("No se encontraron notas en el archivo MIDI.")
        return

    # Preprocesar para el modelo
    input_data = preprocess_notes(notes)

    # Inicializar modelo (esto es un ejemplo; en producción, carga un modelo entrenado)
    model = MusicNet()
    model.eval()
    with torch.no_grad():
        output = model(input_data)
        # Aquí el modelo podría clasificar notas o corregirlas; por ahora, usamos las notas originales

    # Generar partitura con las notas originales (o procesadas por el modelo)
    create_score(notes, bpm, output_pdf, output_xml)

# 6. Ejecutar el programa
if __name__ == "__main__":
    # Reemplaza 'input.mid' con la ruta a tu archivo MIDI
    midi_file = "input.mid"
    if os.path.exists(midi_file):
        transcribe_midi_to_score(midi_file, output_pdf="partitura.pdf", output_xml="partitura.xml")
    else:
        print("Por favor, proporciona un archivo MIDI válido (input.mid).")

MusicXMLExportException: In part (None), measure (29): Cannot convert inexpressible durations to MusicXML.

In [6]:
# 2. Import libraries
import pretty_midi
import numpy as np
import torch
import torch.nn as nn
from music21 import stream, note, meter, key, tempo
import os
from google.colab import files

# 3. Define a simple neural network (example, not trained)
class MusicNet(nn.Module):
    def __init__(self, input_size=4, hidden_size=128, output_size=88):  # 88 piano keys
        super(MusicNet, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return torch.sigmoid(out)

# 4. Load and process MIDI file
def load_midi(file_path):
    try:
        midi_data = pretty_midi.PrettyMIDI(file_path)
        notes = []
        for instrument in midi_data.instruments:
            if not instrument.is_drum:
                for n in instrument.notes:
                    notes.append({
                        'pitch': n.pitch,
                        'start': n.start,
                        'duration': n.end - n.start,
                        'velocity': n.velocity
                    })
        tempos = midi_data.get_tempo_changes()[1]
        bpm = float(tempos[0]) if len(tempos) > 0 else 120.0
        return notes, bpm
    except Exception as e:
        print(f"Error loading MIDI: {e}")
        return [], 120.0

# 5. Preprocess notes for the model
def preprocess_notes(notes):
    data = np.array([[n['pitch'], n['start'], n['duration'], n['velocity']] for n in notes])
    data[:, 0] = (data[:, 0] - 21) / (108 - 21)  # Normalize pitch
    data[:, 3] = data[:, 3] / 127  # Normalize velocity
    data[:, 1] = data[:, 1] / np.max(data[:, 1]) if np.max(data[:, 1]) > 0 else 0
    data[:, 2] = data[:, 2] / np.max(data[:, 2]) if np.max(data[:, 2]) > 0 else 0
    return torch.tensor(data, dtype=torch.float32).unsqueeze(0)

# 6. Quantize duration to nearest standard music21 duration
def quantize_duration(quarter_length):
    """Round quarterLength to the nearest standard duration."""
    standard_durations = [0.125, 0.25, 0.5, 1.0, 2.0, 4.0]  # 32nd, 16th, 8th, quarter, half, whole
    return min(standard_durations, key=lambda x: abs(x - quarter_length))

# 7. Generate score with music21
def create_score(notes, bpm, output_xml="output.xml"):
    score = stream.Score()
    part = stream.Part()
    part.append(key.Key('C'))  # Default key
    part.append(meter.TimeSignature('4/4'))  # Default time signature
    part.append(tempo.MetronomeMark(number=bpm))

    for i, n in enumerate(notes):
        quarter_length = n['duration'] * (bpm / 60)
        if quarter_length < 0.1 or quarter_length > 8.0:  # Avoid extreme durations
            print(f"Warning: Invalid duration {quarter_length} at note {i}, setting to 0.25")
            quarter_length = 0.25
        # Quantize duration to avoid inexpressible values
        quarter_length = quantize_duration(quarter_length)
        try:
            midi_note = note.Note(pitch=n['pitch'], quarterLength=quarter_length)
            midi_note.volume.velocity = n['velocity']
            part.append(midi_note)
        except Exception as e:
            print(f"Error adding note {i} (pitch: {n['pitch']}, duration: {quarter_length}): {e}")
            continue

    score.append(part)

    # Save as MusicXML
    try:
        score.write('musicxml', output_xml)
        print(f"Score saved as {output_xml}")
        return output_xml
    except Exception as e:
        print(f"Error saving MusicXML: {e}")
        return None

# 8. Main function
def transcribe_midi_to_score(midi_file_path, output_xml="partitura.xml"):
    notes, bpm = load_midi(midi_file_path)
    if not notes:
        print("No notes found in MIDI file.")
        return None

    input_data = preprocess_notes(notes)
    model = MusicNet()
    model.eval()
    with torch.no_grad():
        output = model(input_data)  # Example; model not trained

    output_file = create_score(notes, bpm, output_xml)
    return output_file

# 9. Upload MIDI file in Colab
print("Please upload your MIDI file:")
uploaded = files.upload()

# 10. Process uploaded file
if uploaded:
    midi_file = list(uploaded.keys())[0]
    output_xml = "partitura.xml"
    output_file = transcribe_midi_to_score(midi_file, output_xml=output_xml)

    if output_file and os.path.exists(output_file):
        print(f"Downloading {output_file}...")
        files.download(output_file)
    else:
        print("No output file generated.")
else:
    print("No MIDI file uploaded.")

Please upload your MIDI file:


Saving Frank_Sinatra_-_My_Way.mid to Frank_Sinatra_-_My_Way.mid


beam: WARNING: Found a messed up beam pair <music21.beam.Beams <music21.beam.Beam 1/start>/<music21.beam.Beam 2/partial/right>>, <music21.beam.Beams <music21.beam.Beam 1/stop>/<music21.beam.Beam 2/stop>>, at index 4 of 
[None, None, None, None, <music21.beam.Beams <music21.beam.Beam 1/start>/<music21.beam.Beam 2/partial/right>>, <music21.beam.Beams <music21.beam.Beam 1/stop>/<music21.beam.Beam 2/stop>>]
beam: WARNING: Found a messed up beam pair <music21.beam.Beams <music21.beam.Beam 1/start>/<music21.beam.Beam 2/partial/right>>, <music21.beam.Beams <music21.beam.Beam 1/stop>/<music21.beam.Beam 2/stop>>, at index 6 of 
[<music21.beam.Beams <music21.beam.Beam 1/start>/<music21.beam.Beam 2/start>>, <music21.beam.Beams <music21.beam.Beam 1/stop>/<music21.beam.Beam 2/stop>>, None, None, None, None, <music21.beam.Beams <music21.beam.Beam 1/start>/<music21.beam.Beam 2/partial/right>>, <music21.beam.Beams <music21.beam.Beam 1/stop>/<music21.beam.Beam 2/stop>>]
beam: WARNING: Found a messed up

Score saved as partitura.xml


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# 1. Install dependencies in Colab
!pip install pretty_midi music21 torch numpy
!apt-get update
!apt-get install lilypond -y  # Install LilyPond in Colab

# 2. Import libraries
import pretty_midi
import numpy as np
import torch
import torch.nn as nn
from music21 import stream, note, meter, key, tempo
import os
import subprocess
from google.colab import files

# 3. Define a simple neural network (example, not trained)
class MusicNet(nn.Module):
    def __init__(self, input_size=4, hidden_size=128, output_size=88):
        super(MusicNet, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return torch.sigmoid(out)

# 4. Load and process MIDI file
def load_midi(file_path):
    try:
        midi_data = pretty_midi.PrettyMIDI(file_path)
        notes = []
        print("Available tracks:")
        for i, instrument in enumerate(midi_data.instruments):
            print(f"Track {i}: Program {instrument.program}, Name: {instrument.name}, Is drum: {instrument.is_drum}")
            if not instrument.is_drum and instrument.program in range(0, 8):
                for n in instrument.notes:
                    duration = n.end - n.start
                    if duration > 0:
                        notes.append({
                            'pitch': n.pitch,
                            'start': n.start,
                            'duration': duration,
                            'velocity': n.velocity
                        })
        tempos = midi_data.get_tempo_changes()[1]
        bpm = float(tempos[0]) if len(tempos) > 0 else 120.0
        print(f"Detected BPM: {bpm}")
        return notes, bpm
    except Exception as e:
        print(f"Error loading MIDI: {e}")
        return [], 120.0

# 5. Preprocess notes for the model
def preprocess_notes(notes):
    if not notes:
        return None
    data = np.array([[n['pitch'], n['start'], n['duration'], n['velocity']] for n in notes])
    data[:, 0] = (data[:, 0] - 21) / (108 - 21)
    data[:, 3] = data[:, 3] / 127
    data[:, 1] = data[:, 1] / np.max(data[:, 1]) if np.max(data[:, 1]) > 0 else 0
    data[:, 2] = data[:, 2] / np.max(data[:, 2]) if np.max(data[:, 2]) > 0 else 0
    return torch.tensor(data, dtype=torch.float32).unsqueeze(0)

# 6. Quantize duration
def quantize_duration(quarter_length):
    standard_durations = [0.125, 0.25, 0.5, 1.0, 2.0, 4.0]
    triplet_durations = [1/3, 2/3, 4/3]
    all_durations = standard_durations + triplet_durations
    return min(all_durations, key=lambda x: abs(x - quarter_length))

# 7. Generate score with music21
def create_score(notes, bpm, output_xml="output.xml", output_pdf="output.pdf"):
    score = stream.Score()
    part = stream.Part()
    part.append(key.Key('C'))
    part.append(meter.TimeSignature('4/4'))
    part.append(tempo.MetronomeMark(number=bpm))

    for i, n in enumerate(notes):
        quarter_length = n['duration'] * (bpm / 60)
        if quarter_length < 0.1 or quarter_length > 8.0:
            print(f"Warning: Invalid duration {quarter_length} at note {i}, setting to 0.25")
            quarter_length = 0.25
        quarter_length = quantize_duration(quarter_length)
        try:
            midi_note = note.Note(pitch=n['pitch'], quarterLength=quarter_length)
            midi_note.volume.velocity = n['velocity']
            midi_note.beams = None
            part.append(midi_note)
        except Exception as e:
            print(f"Error adding note {i} (pitch: {n['pitch']}, duration: {quarter_length}): {e}")
            continue

    score.append(part)
    part.makeNotation(inPlace=True, cautionaryNotImmediateRepeat=False)
    for n in part.flat.notes:
        n.beams = None

    # Save as MusicXML
    score.write('musicxml', output_xml)
    print(f"Score saved as {output_xml}")

    # Try converting to PDF with LilyPond
    try:
        subprocess.run(["musicxml2ly", output_xml, "-o", "partitura.ly"], check=True)
        subprocess.run(["lilypond", "-o", "partitura", "partitura.ly"], check=True)
        print(f"PDF saved as partitura.pdf")
        return output_xml, "partitura.pdf"
    except Exception as e:
        print(f"Error converting to PDF with LilyPond: {e}. Download {output_xml} and use MuseScore locally.")
        return output_xml, None

# 8. Main function
def transcribe_midi_to_score(midi_file_path, output_xml="partitura.xml", output_pdf="partitura.pdf"):
    notes, bpm = load_midi(midi_file_path)
    if not notes:
        print("No piano notes found in MIDI file.")
        return None, None
    input_data = preprocess_notes(notes)
    if input_data is None:
        print("No data to process for the model.")
        return None, None
    model = MusicNet()
    model.eval()
    with torch.no_grad():
        output = model(input_data)
    output_xml, output_pdf = create_score(notes, bpm, output_xml, output_pdf)
    return output_xml, output_pdf

# 9. Upload MIDI file in Colab
print("Please upload your MIDI file:")
uploaded = files.upload()

# 10. Process uploaded file
if uploaded:
    midi_file = list(uploaded.keys())[0]
    output_xml, output_pdf = transcribe_midi_to_score(midi_file)
    if output_xml and os.path.exists(output_xml):
        print(f"Downloading {output_xml}...")
        files.download(output_xml)
    if output_pdf and os.path.exists(output_pdf):
        print(f"Downloading {output_pdf}...")
        files.download(output_pdf)
else:
    print("No MIDI file uploaded.")

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.2 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,963 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:14 h

Saving brahms-lullaby-wiegenlied-piano.mid to brahms-lullaby-wiegenlied-piano.mid
Available tracks:
Track 0: Program 0, Name: Piano, Is drum: False
Detected BPM: 80.0


/usr/local/lib/python3.12/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


Score saved as partitura.xml
PDF saved as partitura.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>